In [2]:
import sys, os
module_path="../.."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

chunk_size = 26
chunk_overlap = 4

r_splitter = RecursiveCharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap
)

### RecusrsiveCharacterTextSplitter 이해

In [4]:
# Recursive text Splitter
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

- 문자의 개수가 26개 이상이기에 4개의 문자를 중복(overap)시키고, 두개로 분리

In [5]:
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

- space를 포함해서 최대 26개 글자씩 분리

In [7]:
# Recursive text Splitter
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)


['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

### CharacterTextSplitter의 이해
Separator가 없는 CharaterTextSplitter은 26개의 chunk size는 무시하고 전체(51)를 1개의 청크로 사용한다.

In [12]:
# Character Text Splitter
print("length of text3: ", len(text3))
c_splitter.split_text(text3)


length of text3:  51


['a b c d e f g h i j k l m n o p q r s t u v w x y z']

Separator를 space로 주게 되면 이를 분리자로해서 26개 chunk size 이내로 chunk를 분리한다.

In [13]:
c_splitter = CharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap,
  separator=' '
)
c_splitter.split_text(text3)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [14]:
## 2. 실제 문장 이해하기

In [15]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space. \nThis is the last sentence"""

print_ww("some_text: \n\n", some_text)
print("\nlen: ", len(some_text))

some_text:

 When writing documents, writers will use document structure to group content. This can convey to
the reader, which idea's are related. For example, closely related ideas are in sentances. Similar
ideas are in paragraphs. Paragraphs form a document.

 Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are
the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,
have a space.and words are separated by space.
This is the last sentence

len:  522


### RecursiveCharacterTextSplitter
- 전체 글자수가 522인데, chunk_size=450이어서, 450글자 안에서 "\n\n", "\n", " ", ""의 순서대로 분리자를 선택해서 분절한다. 그래서
paragraphs로 시작하는 곳에서 두번째 청크가 시작이 됨

In [18]:
r_splitter = RecursiveCharacterTextSplitter(
  chunk_size=450,
  chunk_overlap=0,
  separators=["\n\n", "\n", " ", ""]
)
docs = r_splitter.split_text(some_text)
print("len(docs): ", len(docs))
print_ww("docs: ", docs)

len(docs):  2
docs:  ["When writing documents, writers will use document structure to group content. This can
convey to the reader, which idea's are related. For example, closely related ideas are in sentances.
Similar ideas are in paragraphs. Paragraphs form a document.", 'Paragraphs are often delimited with
a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded
in this string. Sentences have a period at the end, but also, have a space.and words are separated
by space. \nThis is the last sentence']


In [19]:
c_splitter = CharacterTextSplitter(
  chunk_size=450,
  chunk_overlap=0,
  separator=' '
)
docs = c_splitter.split_text(some_text)
print("len(docs): ", len(docs))
print_ww("docs: ", docs)

len(docs):  2
docs:  ['When writing documents, writers will use document structure to group content. This can
convey to the reader, which idea\'s are related. For example, closely related ideas are in
sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often
delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you
see embedded in this string. Sentences have a period at the end, but also,', 'have a space.and words
are separated by space. \nThis is the last sentence']


## 3. 실제 긴 테스트 문서 (State of Union)로 이해하기

In [24]:
# This is a logn document we can split up.
with open('../data/state_of_the_union.txt') as f:
  state_of_the_union = f.read()

### RecursiveCharacterTextSpliter
- chunk_size == 1000으로 분절, 1000 이내로 자르기 위해 "\n\n", "\n", " ", "" 순서 기준으로 먼저 오는 것을 분절하고, 다음 청크로 분리

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=0,
  separators=["\n\n", "\n", " ", ""],
  length_function=len,
  is_separator_regex=False,
)

texts = text_splitter.create_documents([state_of_the_union])

from utils.rag import show_context_used
show_context_used(texts, limit=5)

### CharacterTextSplitter
- chunk_size = 500과 분리자를 space로 해서 분리

In [ ]:
text_splitter = CharacterTextSplitter(
  chunk_size=500,
  chunk_overlap=0,
  separator=' '
)

texts = text_splitter.create_documents([state_of_the_union])
show_context_used(texts)